In [ ]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import commonteamroster
from nba_api.stats.endpoints import teamgamelog
from nba_api.stats.endpoints import commonteamroster
from nba_api.stats.endpoints import leaguestandingsv3
from datetime import datetime, timezone, timedelta
from dateutil import parser
from nba_api.live.nba.endpoints import scoreboard
from ..Functions.helpfuncs import get_current_season
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple, Union, Optional

pd.set_option("display.max_columns", None)


In [5]:
class Games:
    def __init__(self):
        self.current_season = get_current_season()

    def check_valid_season(self, season: str) -> str:
        """Check if the provided season is valid and return formatted season."""
        if not season:
            season = self.current_season
        elif not season.isdigit():
            raise ValueError(f"Invalid season format: {season}. Expected a year in 'YYYY' format.")
        elif int(season) < 1980 or int(season) > datetime.now().year:
            raise ValueError(f"Invalid season year: {season}. Year must be between 1980 and {datetime.now().year}.")
        else:
            season = f"{season}-{str(int(season)+1)[-2:]}"
        
        return season

    def get_team_game_log(self, team_id: int, season: str) -> pd.DataFrame:
        game_log = teamgamelog.TeamGameLog(team_id=team_id, season=season)
        game_log = game_log.get_data_frames()[0]
        return game_log

    def get_todays_games(self) -> None:
        f = "{gameId}: {awayTeam} @ {homeTeam} : {gameTimeLTZ}" 
        board = scoreboard.ScoreBoard()
        print("ScoreBoardDate: " + board.score_board_date)
        games = board.games.get_dict()
        for game in games:
            gameTimeLTZ = parser.parse(game["gameTimeUTC"]).replace(tzinfo=timezone.utc).astimezone(tz=None)
            print(f.format(gameId=game['gameId'], awayTeam=game['awayTeam']['teamName'], homeTeam=game['homeTeam']['teamName'], gameTimeLTZ=gameTimeLTZ))

    def get_current_standings(self, season: str = None, conference:str = 'Overall') -> pd.DataFrame:
        """Get current NBA standings for all teams"""
        conference_types = ['Overall','West','East']
        try:
            if conference in conference_types:
                season = self.check_valid_season(season)
                
                standings = leaguestandingsv3.LeagueStandingsV3(
                    league_id='00',  # NBA
                    season=season,
                    season_type='Regular Season'
                )
                
                standings_df = standings.get_data_frames()[0]
                
                # Select relevant columns
                columns_to_keep = [
                    'TeamID', 'TeamCity', 'TeamName', 'Conference','WINS', 'LOSSES', 'PlayoffRank', 
                    'Record', "HOME","ROAD", "L10"
                ]
                
                standings_df = standings_df[columns_to_keep]

                if conference != 'Overall':
                    return standings_df[standings_df['Conference'] == conference]
                else:
                    return standings_df
            else:
                raise ValueError(f"Invalid conference type: {conference}. Expected one of {conference_types}.")    
        except Exception as e:
            print(f"Error retrieving standings for season {season}: {e}")
            raise e


In [6]:
Games().get_current_standings(season='2025', conference='East')

,TeamID,TeamCity,TeamName,Conference,WINS,LOSSES,PlayoffRank,Record,HOME,ROAD,L10
1,1610612765,Detroit,Pistons,East,15,3,1,15-3,7-1,7-2,9-1
2,1610612761,Toronto,Raptors,East,14,5,2,14-5,7-2,7-3,9-1
4,1610612748,Miami,Heat,East,13,6,3,13-6,9-1,4-5,8-2
7,1610612752,New York,Knicks,East,11,6,4,11-6,8-1,3-5,7-3
8,1610612739,Cleveland,Cavaliers,East,12,7,5,12-7,8-3,4-4,6-4
11,1610612737,Atlanta,Hawks,East,11,8,6,11-8,3-4,8-4,7-3
12,1610612753,Orlando,Magic,East,11,8,7,11-8,7-3,4-5,7-3
14,1610612738,Boston,Celtics,East,10,8,8,10-8,6-4,4-4,7-3
16,1610612741,Chicago,Bulls,East,9,8,9,9-8,6-2,3-6,3-7
18,1610612755,Philadelphia,76ers,East,9,8,10,9-8,5-5,4-3,4-6
